# On particle filters applied to electricity load forecasting

PyMC tuto

https://pymc-devs.github.io/pymc/tutorial.html#fitting-the-model-with-mcmc

http://sdsawtelle.github.io/blog/output/mcmc-in-python-with-pymc.html
    

NC Email:

Voilà quelques pistes que vous pouvez suivre:
* commencer par implémenter un bootstrap filter pour le modèle proposé, et des valeurs des paramètres choisis à la main ﴾en fonction des valeurs reportées dans l'article par exemple﴿ et des données que vous pouvez simuler. Déterminer à quel point l'algorithme marche bien:
+ graphe de l'ESS en fonction du temps; voir si effectivement certains outliers font fortement chuter l'ESS. Essayer la méthode proposée pour gérer ses outliers.
+ intervalles de confiance pour l'estimation de E[X_t|Y_{0:t}], basé sur plusieurs exécutions, par exemple.

Deuxième étape: estimation des paramètres. Vous pouvez essayer une des trois approches suivantes, triées par ordre de difficulté:
1. maximisation de la vraisemblance ﴾estimée par votre filtre particulaire pour un theta donné﴿, en utilisant une méthode de maximisation pour fonction bruitée ﴾par ex Robbins‐Monroe﴿.
2. PMMH, voir le prochain cours. Principale difficulté: l'algorithme peut prendre du temps, et ne pas être très facile à calibrer, mais je vais en parler en cours.
3. Particle Gibbs: c'est peut‐être un peu compliqué pour ce modèle, mais si vous arrivez, gros bonus!

In [1]:
import pandas as pd
import numpy as np
import numpy.random as npr
import matplotlib as mtp
import matplotlib.pyplot as plt
import scipy as sp
from scipy.stats import truncnorm
import scipy.stats as stats
from scipy.stats import invgamma
from scipy import ndimage
import math

### Parameters

M=10**3 #number of particles
n_pred=365  #number of predictions
N_daytype=8  #number of day types

### Load temperatures data 2015-2016

df_temp=pd.read_csv('data/temp_1516.csv',sep=',')
df_temp=df_temp.drop(df_temp.columns[0],axis=1)
df_temp=df_temp.sort_values(by='date')
df_temp.head()

#temperatures every day at 3pm
temp_day=df_temp[df_temp.hour=="15:00"]
temp_day.head()
#Array vector of temperatures for each day of the year
T_h=list(temp_day.t)

### Load electricity data

mypath="data/"
df=pd.read_csv(mypath+"cdc_conso_daytypes.csv",sep=",",encoding='latin-1')
df=df.drop(["daytype"+str(i) for i in range(0,9)],axis=1)
df=df.drop(df.columns[0],axis=1)
df.index=range(0,len(df))
df=df.sort_values(by=['Date'])
df['Date - Heure']=pd.to_datetime(df['Date - Heure'])
df.sort_values(by=["Date - Heure"])
#truncate to the data from 2015
df=df[(df.Date >='2015-01-01')]
df.head()

plt.plot_date(df.Date,df['Consommation (MW)'],fmt='-')
plt.show()

bins=range(30000,100000,1000)
plt.hist(df['Consommation (MW)'],normed=True,bins=bins)
plt.show()

### Set variables of electricity demand

from datetime import date
df['Consommation']=df['Consommation (MW)']   #leave it in MW
#For prediction, every day at 3pm
consumption_day_ahead = list(df[df['Heure']=='15:00']['Consommation'])
#for initialization
consumption = list(df[df['Heure']=='15:00']['Consommation'][:30])

## Implementation of Algorithm 3.10

## At time n=0:

### 1.Definition of MCMC initial model at n=0 - sample $X_0^{j} \sim \mu(x_0)$

### Option A: Set values by hand

#Add a bit of noise with uniform random
#s=8000*np.ones(M)*npr.uniform(0.5,1.5,size=M)
#sigma2_g_star=(10**5)*npr.uniform(0.5,1.5,size=M)
#sigma2_s_star=(10**5)*npr.uniform(0.5,1.5,size=M)
sigma_g_init=1.27*10**(8/2)
sigma_s_init=(2*7.9)*10**3
u_h=14 #mean of N(14,1)
kappa=np.array([1/9]*9)
sigma2=2.7*10**7
g_heat_init = stats.truncnorm.rvs(-np.inf,0,loc=0,scale=10**4,size=M)
s_init = stats.truncnorm.rvs(np.inf,0,loc=0,scale=10**4,size=M)
sigma_s_star_2_evol=7.9*10**7
sigma_g_star_2_evol=1.27*10**8

x_init=x_season(df.daytype[15],kappa,s_init,sigma_s_init)[0]+x_heat(g_heat_init,15,sigma_g_init)[0]
w_init= np.exp(-(np.square(consumption[15]-x_init))/(2*sigma2))

### Option C: MCMC using MHA or Gibbs (TBA)

#### Freeze parameters from Zak's simulation in gibbs-parameters_init_v1

N_daytype=9
k_day=npr.dirichlet(np.ones(N_daytype),1)
len_init=15
pred_forward=[0,1,2]

#Load pickle file
import pickle
output_file ='data/parameters_init_20180113-135329.pkl'
pkl_file = open(output_file, 'rb')
parameters_init = pickle.load(pkl_file)
parameters_init["x_init"][:20]

parameters_init

s_init=parameters_init['s_init']
g_heat_init=parameters_init['g_heat_init']
sigma_s_init=parameters_init['sigma_s_init']
sigma_g_init=parameters_init['sigma_g_init']
x_init=parameters_init['x_init']
sigma2=parameters_init['sigma2']
u_h=parameters_init['u_h']
kappa=parameters_init['kappa']
w_init=parameters_init['w_init']

sigma2=np.ones(M)*3*10**7
sigma_g_init=np.ones(M)*10**4
sigma_s_init=np.ones(M)*10**4
kappa=np.ones(M)*1/9

w_init[0]=np.median(w_init)
x_init[0]=np.median(x_init)
g_heat_init[0]=np.median(g_heat_init)
s_init[0]=np.median(s_init)

### Part 2: regularize weights and x if necessary

#STEP 2 OF PARTICLE FILTER
def resample(x_pred,w_prev,nbdays_pred_today,len_init,n,sigma_s,sigma_g,g_h,s,sigma,ESS):   
    #compute y_n
    delta_cons_gaus=-np.square(consumption_day_ahead[len_init+n+nbdays_pred_today]-x_pred)/(2*sigma**2)
    y_n=np.exp(delta_cons_gaus)
    #compute new weights
    if n>0:
        w_=w_prev*y_n
    else:
        w_=w_prev
    #likelihood of y_n
    lh_y_n=np.sum(delta_cons_gaus)
    #normalize weights
    w_h=w_/sum(w_)
    #calculate ESS
    ESS=1/sum(np.square(w_h))
    x =np.zeros(M)
    w =np.zeros(M)
    print("ESS of normalized weights=",round(ESS,6))
    if ESS <0.001*M: #reset the weights, keep x predicted as such
        print("ESS <0.001*M")
        x=x_pred
        if n==0:
            w=np.ones(M)*(1/M)
        w=w_prev
    elif (ESS>=0.001*M and ESS<0.5*M):  #reset all the weights, add some noise to a fraction of the x's
        print("ESS>=0.001*M and ESS_0<0.5*M")
        x,w,sigma_s,sigma_g,g_h,s=resample_multinomial(x_pred,w_h,sigma_s,sigma_g,g_h,s)
    elif ESS>=0.5*M:  #No degeneracy
        print("ESS>=0.5*M")
        x=x_pred
        w=w_h
    else:
        print("ESS critically low")
        x=x_pred
        if n==0:
            w=np.ones(M)*(1/M)
        w=w_prev
            
    print("new ESS=",round(1/sum(np.square(w)),6))
    return x,w,ESS,lh_y_n,sigma_s,sigma_g,g_h,s

def resample_multinomial(x_temp,w_temp,sigma_s,sigma_g,g_h,s):
    multinomial = np.random.multinomial(1,w_temp,M)
    new_x = np.zeros(M)
    new_s = np.zeros(M)
    new_g_heat = np.zeros(M)
    new_sigma_s = np.zeros(M)
    new_sigma_g = np.zeros(M)
    for i in range(M):
        new_x[i]=x_temp[np.argmax(multinomial[i,])]
        new_s[i]=s[np.argmax(multinomial[i,])]
        new_g_heat[i]=g_h[np.argmax(multinomial[i,])]
        new_sigma_s[i]=sigma_s[np.argmax(multinomial[i,])]
        new_sigma_g[i]=sigma_g[np.argmax(multinomial[i,])]
    new_w=(1/M)*np.ones(M)   
    return new_x,new_w,new_sigma_s,new_sigma_g,new_g_heat,new_s

#initialize matrix of x_heat, x_season
ESS=np.array(np.ones(n_pred+1))
x =np.zeros([n_pred+1,M])
w =np.zeros([n_pred+1,M])
lh_y_n =np.zeros(n_pred+1)
x_season =np.zeros([n_pred+1,M]) 
x_heat =np.zeros([n_pred+1,M])

x[0,:],w[0,:],ESS[0],lh_y,sigma_s_init,sigma_g_init,g_heat_init,s_init=resample(x_init,w_init,2,15,0,sigma_s_init,
                                                                                sigma_g_init,
                                                                                g_heat_init,s_init,sigma2**0.5,ESS)

## Prediction and Filtering at time n>0

### 1. Sample $x^j_n \mid X^j_{n-1} $ for all j=1...M particles

def x_season(day_type,k_day,s_prev,sigma_s_prev):
    nu=truncnorm.rvs(a = (-sigma_s_prev-0) / sigma_g_init , b = np.inf, loc= 0, scale = sigma_g_init, size=M)
    sigma_s=sigma_s_prev+nu
    err=truncnorm.rvs(a = -s_prev / sigma_s , b = np.inf, loc= 0, scale = sigma_s, size=M)
    s=s_prev+err
    x_s=s*k_day[day_type]
    return x_s, s, sigma_s

def x_heat(g_h_prev,day,sigma_g_prev):
    nu=truncnorm.rvs(a = -sigma_g_prev / sigma_g_init , b = np.inf, loc= 0, scale = sigma_g_init, size=M)
    sigma_g=sigma_g_prev+nu
    err=truncnorm.rvs(a = -np.inf , b =(-g_h_prev-0) / sigma_g, loc= 0, scale = sigma_g, size=M)
    g_h=g_h_prev+err
    if(u_h-T_h[day]<0):
        print("No heating effect")
    x_h=g_h*(T_h[day]-u_h)*max(np.sign(u_h-T_h[day]),0)
    return x_h, g_h, sigma_g

daytype=np.array(df.daytype)

print(x_season(daytype[15],kappa,s_init,sigma_s_init)[2])
print(x_heat(g_heat_init,16,sigma_g_init)[2])

#Initialize parameters
def particle_filter(nbdays_pred_today,len_init,len_filtering,s,g_h,sigma_s,sigma_g,sigma,lh_y):
    lh_y_n=np.zeros(len_filtering)
    x_pred=np.zeros([len_filtering,M])
    x_pred_mean=np.zeros(len_filtering)
    ESS=np.zeros(len_filtering)
    for n in range(1,len_filtering):
        print("n=",n)
        #prediction X[n] one day ahead, hourly forecast
        x_s=x_season(int(daytype[len_init+n+nbdays_pred_today]),kappa,s,sigma_s)
        x_h=x_heat(g_h,n+len_init+nbdays_pred_today,sigma_g)
        x_pred[n,:]=x_s[0]+x_h[0]
        #print("number of negative values:",len(x_pred[x_pred<0]))
        print("x_pred_mean =","{:.2e}".format(np.mean(x_pred[n,:])),
              "real consumption=","{:.2e}".format(consumption_day_ahead[n]))
        print("x_pred min=","{:.2e}".format(np.min(x_pred[n,:])),"x_pred max","{:.2e}".format(np.max(x_pred[n,:])))
        #take new values of parameters to feed x_season and x_heat in the next step
        s, sigma_s=x_s[1:]
        g_h, sigma_g=x_h[1:]
        #regularization
        x[n,:],w[n,:],ESS[n],lh_y_n[n],sigma_s,sigma_g,g_h,s=resample(x_pred[n,:],w[n-1,:],nbdays_pred_today,len_init,n,sigma_s,sigma_g,g_h,s,sigma,ESS)
        print("------------------------")
        x_pred_mean[n]=np.mean(x_pred[n,:])
    return lh_y_n,x_pred_mean,ESS

x_predict=np.zeros([len(pred_forward),n_pred])
ESS_calc=np.zeros([len(pred_forward),n_pred])
for i in range(len(pred_forward)):
    log_lh_init,x_predict[i,:],ESS_calc[i,:]=particle_filter(pred_forward[i],len_init,n_pred,s_init,g_heat_init,sigma_s_init,sigma_g_init,sigma2**0.5,lh_y_n)

fig=plt.figure(figsize=(12,6))
plt.plot(range(1,n_pred),ESS_calc[0,1:n_pred]/M)
plt.plot(range(1,n_pred),ESS_calc[1,1:n_pred]/M)
plt.plot(range(1,n_pred),ESS_calc[2,1:n_pred]/M)

plt.title("Evolution of ESS",fontweight='bold')
plt.xlabel('forecasted day')
plt.xlim(0,n_pred)
plt.show()

fig=plt.figure(figsize=(12,6))


plt.plot(range(n_pred),(x_predict[0,:]-consumption_day_ahead[:n_pred])/consumption_day_ahead[:n_pred],color='blue')
plt.plot(range(n_pred),(x_predict[1,:]-consumption_day_ahead[1:n_pred+1])/consumption_day_ahead[1:n_pred+1],color='orange')
plt.plot(range(n_pred),(x_predict[2,:]-consumption_day_ahead[2:n_pred+2])/consumption_day_ahead[2:n_pred+2],color='green')

plt.plot(range(n_pred-1),np.zeros(n_pred-1),color='red',linestyle='--')
plt.ylim(-1,1)
plt.xlim(0,n_pred)
plt.xlabel('forecasted day')
plt.title("Relative forecast error",fontweight='bold')
plt.show()

fig=plt.figure(figsize=(12,6))
plt.plot(range(n_pred),x_predict[0,:])
plt.plot(range(n_pred),x_predict[1,:])
plt.plot(range(n_pred),x_predict[2,:])
plt.plot(range(n_pred),consumption_day_ahead[:n_pred],linestyle='--')
plt.xlabel('forecasted day')
plt.ylim(10**4,10**5)
plt.title("Electricity load forecast in W",fontweight='bold')
plt.show()

Goal: re-estimate the one-dimensional parameters ($\sigma_g,\sigma_s,\mu_h,\kappa,\sigma$)<br>
Proposal distribution for these parameters: truncated gaussian random walk<br>
We set kappa constant at 1/8



## PMMH

#### Initialization of parameters

u_h_current=13
sigma_current=10**4
sigma_g_current=10**4
sigma_s_current=10**4

len_filter_pmmh=365
len_iter_mha=2
accept_log_proba=np.zeros(len_filter_pmmh)
accept_rate=0
log_lh_init=np.zeros(len_filter_pmmh)
lh_y_prop=np.zeros(len_filter_pmmh)

#store the accepted value in lists
u_h_list=[]
sigma_list=[]
sigma_g_list=[]
sigma_s_list=[]

#### Initialization of hyperparameters

#standard deviation of normal/trunc normal proposals on parameters
std_hyp_sigma_g,std_hyp_sigma_s,std_hyp_sigma=np.ones(3)*5*10**3
std_hyp_u=1

#joint prior density of parameters
def log_joint_prior(u_h,sigma,sigma_g,sigma_s):
    res=0
    res=(-(u_h-14)**2)/2
    res=res+(-0.01-1)*np.log(sigma**2) - (0.01/sigma**2)
    res=res+(-0.01-1)*np.log(sigma_g**2) - (0.01/sigma_g**2)
    res=res+(-0.01-1)*np.log(sigma_s**2) - (0.01/sigma_s**2)
    return res

#joint log prior density initialize
log_prior_init=log_joint_prior(u_h_current,sigma_current,sigma_g_current,sigma_s_current)
print(log_prior_init)
#initial parameters otbained from Gibbs. These initialized parameters will not change through iterations

#### Run initial particle filter and get the log likelihood

log_lh_init=particle_filter(pred_forward[0],len_init,len_filter_pmmh,s_init,g_heat_init,sigma_s_current,sigma_g_current,sigma_current,lh_y_n)[0]

print(log_lh_init[len_filter_pmmh-1])

#### PMMH Algorithm

for step in range(len_iter_mha):
    print("___________________________________________________________")
    print("Metropolis Hastings step:",step)
    #we need 6 inputs to compute the (log) acceptance probability log(r):
    #log_likelihood, joint prior density, log proposal density for both current parameters and proposed parameters
    #sample proposal for u_h, sigma, sigma_g, sigma_s
    u_h_prop=npr.normal(u_h_current,std_hyp_u,size=1)
    sigma_prop=stats.truncnorm.rvs(a=(0-sigma_current)/std_hyp_sigma,b=np.inf,scale=std_hyp_sigma,size=1)
    sigma_g_prop=stats.truncnorm.rvs(a=(0-sigma_g_current)/std_hyp_sigma_g,b=np.inf,scale=std_hyp_sigma_g,size=1)
    sigma_s_prop=stats.truncnorm.rvs(a=(0-sigma_s_current)/std_hyp_sigma_s,b=np.inf,scale=std_hyp_sigma_s,size=1)
    print("proposed parameters:","u_heat:",u_h_prop,"sigma:",sigma_prop,"sigma_g:",sigma_g_prop,"sigma_s:",sigma_s_prop)
    #1/run a particle filter with the proposed parameters to obtain a an estimation of likelihood proposed
    #  consider the likelihood of the last day of the fitering
    lh_y_prop=particle_filter(pred_forward[0],len_init,len_filter_pmmh,s_init,g_heat_init,sigma_s_prop,sigma_g_prop,sigma_prop,lh_y_n)[0]
    print("log likelihood proposal of y:",np.sum(lh_y_prop))
    #2/generate prior proposals and compute joint log density of them
    log_prior_prop = log_joint_prior(u_h_prop,sigma_prop,sigma_g_prop,sigma_s_prop)
    print("proposed log prior:",log_prior_prop)
    #3/compute log proposal density h(current parameter|proposed parameter)
    current_log_density=np.log(stats.norm.cdf(sigma_current/std_hyp_sigma,loc=0,scale=1))+np.log(
        stats.norm.cdf(sigma_s_current/std_hyp_sigma_s,loc=0,scale=1))+np.log(
        stats.norm.cdf(sigma_g_current/std_hyp_sigma_g,loc=0,scale=1))
    print("proposal log density initial parameters given proposed param:",current_log_density)
    #4/log likelihood from initial parameters --> already done: log_lh_init
    #5/joint prior of the initial parameters: we already have them
    #6/compute log proposal density h(proposed parameter|current parameter)
    prop_log_density=np.log(stats.norm.cdf(sigma_prop/std_hyp_sigma,loc=0,scale=1))+np.log(
    stats.norm.cdf(sigma_s_prop/std_hyp_sigma_s,loc=0,scale=1))+np.log(
    stats.norm.cdf(sigma_g_prop/std_hyp_sigma_g,loc=0,scale=1))
    print("proposal log density proposed parameters given current param:",prop_log_density)
    #we add up these elements to construct the log acceptance probability
    #numerator
    accept_log_proba[step]=np.sum(lh_y_prop)+log_prior_prop+current_log_density
    #denominator
    accept_log_proba[step]=accept_log_proba[step]-np.sum(log_lh_init)-log_prior_init-prop_log_density
    print("acceptance log probability:",accept_log_proba[step])
    u=npr.random()
    #to get an acceptance rate > 5%, we need log_proba to be at least -3
    if(np.log(u)<min(0,accept_log_proba[step])):
        print("ACCEPT")
        accept_rate=accept_rate+1
        log_lh_init=lh_y_prop
        sigma_current=sigma_prop
        sigma_g_current=sigma_g_prop
        sigma_s_current=sigma_s_prop
        u_h_current=u_h_prop
        #store the accepted values
        u_h_list.append(u_h_current)
        sigma_list.append(sigma_current)
        sigma_g_list.append(sigma_g_current)
        sigma_s_list.append(sigma_s_current)
    else:
        print("REJECT")

print(accept_rate/len_iter_mha)

print(sigma_current)
print(sigma_g_current)
print(sigma_s_current)
print(u_h_current)

